In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# Utility Methods

In [ ]:
def display_images(img_arr):
    """ Displays a horizontal row of grayscale images. """
    
    f, axarr = plt.subplots(nrows=1, ncols=len(img_arr), figsize=(10, 10))
    
    for i, img in enumerate(img_arr):
        plt.sca(axarr[i])
        plt.axis('off')
        plt.imshow(img_arr[i], cmap='gray')
   
    plt.show()

In [ ]:
def pad_image(inp, border):
    """ Adds padding of size = border to every edge of the image. """
    
    x, y = inp.shape
    out = np.zeros((x + (border * 2), y + (border * 2)), dtype=int)
    out[border:-border, border:-border] = inp[:,:]
    return out

In [ ]:
def apply_filter(inp, filtr):
    """Applies a specified filter to an image."""
    
    x, y = inp.shape
    border = filtr.shape[0] // 2
    padded = pad_image(inp, border)

    out = np.zeros([x, y])
    for i in range(border, x + border):
        for j in range(border, y + border):
            section = padded[i - border : i + border + 1, j - border : j + border + 1]
            out[i - border, j - border] = np.sum(np.multiply(section, filtr))
    
    return out.astype(np.uint8)

# Task 01

In [ ]:
def box_filter(inp, filter_size):
    """ A filter of the specified size containing the value 1/filter_size is applied. """
    
    filtr = np.ones([filter_size, filter_size], dtype = int) / (filter_size * filter_size)
    return apply_filter(inp, filtr)

In [ ]:
inp = cv2.imread("./Fig0333(a)(test_pattern_blurring_orig).tif", cv2.IMREAD_GRAYSCALE)
filtered_1 = box_filter(inp, filter_size=3)
filtered_2 = box_filter(inp, filter_size=7)
filtered_3 = box_filter(inp, filter_size=15)
display_images([inp, filtered_1, filtered_2, filtered_3])

As can be seen, a larger filter size causes the image to become blurrier. Noise is reduced to a larger extent.

In [ ]:
def weighted_filter(inp, filter_size):
    """ A weighted filter, where the weights are incrementing from the edges of to the center, is applied. """
    
    mid = filter_size // 2
    filtr = np.ones([filter_size, filter_size], dtype = int)
    
    for i in range(mid + 1):
        row = np.ones([1, filter_size])
        for j in range(mid + 1):
            row[0, j] = i + j + 1
            row[0, filter_size - j - 1] = i + j + 1
        filtr[i] = row
        filtr[filter_size - i - 1] = row
    
    filtr = filtr / np.sum(filtr)
    return apply_filter(inp, filtr)

In [ ]:
inp = cv2.imread("./Fig0333(a)(test_pattern_blurring_orig).tif", cv2.IMREAD_GRAYSCALE)
filtered_1 = weighted_filter(inp, filter_size=3)
filtered_2 = weighted_filter(inp, filter_size=7)
filtered_3 = weighted_filter(inp, filter_size=15)
display_images([inp, filtered_1, filtered_2, filtered_3])

The effect of the filter size is similar for weighted filters as it is for box filters. As the filter size increases, the image gets blurried and noise is reduced to a larger extent.

In [ ]:
inp = cv2.imread("./Fig0333(a)(test_pattern_blurring_orig).tif", cv2.IMREAD_GRAYSCALE)
filtered_1 = box_filter(inp, filter_size=15)
filtered_2 = weighted_filter(inp, filter_size=15)
display_images([filtered_1, filtered_2])

Comparing the effects of a box filter and a weighted average filter side by side shows that a box filter has a stronger smoothing effect for the same filter size.

# Task 02

In [ ]:
def add_noise(inp, strength):
    """ 1 pixel of the image is replaced with the value 255 and 1 pixel is replaced with the value 0 randomly 1000 * strength times. """
    
    x, y = inp.shape
    output = inp.copy()
    
    for i in range(strength * 1000):
        nx = random.randint(0, x - 1)
        ny = random.randint(0, y - 1)
        output[nx, ny] = 255
        nx = random.randint(0, x - 1)
        ny = random.randint(0, y - 1)
        output[nx, ny] = 0
    
    return output

In [ ]:
def median_filter(inp, filter_size):
    """ The image is parsed section by section, with the section size specified by the filter size, and the central pixel of each section is replaced with the median value for the section. """
    
    x, y = inp.shape
    border = filter_size // 2
    padded = pad_image(inp, border)
    out = np.zeros([x, y])
    
    for i in range(border, x + border):
        for j in range(border, y + border):
            section = padded[i - border : i + border + 1, j - border : j + border + 1]
            out[i - border, j - border] = np.median(section)
    
    return out.astype(np.uint8)

In [ ]:
inp = cv2.imread("./Fig0335(a)(ckt_board_saltpep_prob_pt05).tif", cv2.IMREAD_GRAYSCALE)
noisy = add_noise(inp, 20)
filtered = median_filter(noisy, 3)
display_images([inp, noisy, filtered])

In [ ]:
inp = cv2.imread("./Fig0335(a)(ckt_board_saltpep_prob_pt05).tif", cv2.IMREAD_GRAYSCALE)
noisy = add_noise(inp, 50)
filtered = median_filter(noisy, 3)
display_images([inp, noisy, filtered])

In [ ]:
inp = cv2.imread("./Fig0335(a)(ckt_board_saltpep_prob_pt05).tif", cv2.IMREAD_GRAYSCALE)
noisy = add_noise(inp, 20)
filtered = median_filter(noisy, 7)
display_images([inp, noisy, filtered])

In [ ]:
inp = cv2.imread("./Fig0335(a)(ckt_board_saltpep_prob_pt05).tif", cv2.IMREAD_GRAYSCALE)
noisy = add_noise(inp, 50)
filtered = median_filter(noisy, 7)
display_images([inp, noisy, filtered])

Analysing the above results shows that a median filter is able to reduce noise in an image without making the edges blurrier. However, if the amount of noise is too large, a median filter of a fixed size is unable to remove all the noise. A larger median filter is required in this situation.

# Task 03

In [ ]:
def laplacian_response(inp):
    """ The laplacian filter, as specified below, is applied to the image. """
    
    filtr = np.array([[-1, -1, -1], 
                     [-1, 8, -1], 
                     [-1, -1, -1]])
    return cv2.filter2D(src=inp, ddepth=-1, kernel=filtr)

In [ ]:
def equalize(inp):
    """ Histogram equalization is applied to the image. """
    
    counts, bins = np.histogram(inp, 256)
    cdf = counts.cumsum()
    cdf = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
    out = np.interp(inp, bins[:-1], cdf)
    out = np.array(out, np.int32)
    return out

In [ ]:
inp = cv2.imread("./Fig0338(a)(blurry_moon).tif", cv2.IMREAD_GRAYSCALE)
laplacian = laplacian_response(inp)

In [ ]:
c = 1
display_images([inp, equalize(laplacian), inp + c * laplacian])

In [ ]:
c = 3
display_images([inp, equalize(laplacian), inp + c * laplacian])

In [ ]:
c = 7
display_images([inp, equalize(laplacian), inp + c * laplacian])

The results above show that increasing the value of c causes increased sharpness in the image. However, increasing it too far causes noise to become more visible.